Вариант 11

Особь - битовая последовательность размера n=200 (кол-во грузов)

Начальная популяция - 1.1 случайная генерация

Отбор - 2.2 выбрать только 20% самых приспособленных особей

Скрещивание - 3.1 многоточечный с 3мя точками

Мутация - 4.2 случайное изменение 3х битов у 5% особей

Формирование новой популяции - 5.2 «штраф» за «старость» - 10% функции приспособленности, выбор лучших

Оценка результата Наступила сходимость (функция приспособленности лучшей особи в популяциях отличается не более, чем на стоимость самой дешевой вещи) или прошло 500 поколений

In [54]:
from random import randint
import copy
import random 
import pandas as pd


In [55]:
myList=[]
with open('11.txt') as f:
    for line in f:
        myList.append([float(x) for x in line.split()])
lngth=len(myList)

maxWeight=myList[0][0]
maxVolume=myList[0][1]

value=[]
size=[]
weight=[]

for i in range (1,lngth):
    #ценность
    value.append(myList[i][2])
    #размер
    size.append(myList[i][1])
    #вес
    weight.append(myList[i][0])

print(myList,'\n')

[[13000.0, 12.0], [205.0, 0.8, 173.0], [489.0, 0.4, 172.0], [1230.0, 1.0, 344.0], [859.0, 0.6, 173.0], [1015.0, 0.9, 333.0], [1524.0, 1.0, 250.0], [1643.0, 0.6, 338.0], [1541.0, 0.7, 324.0], [1010.0, 0.5, 359.0], [1465.0, 1.2, 199.0], [1571.0, 1.0, 350.0], [1092.0, 0.9, 277.0], [470.0, 1.0, 301.0], [959.0, 0.9, 137.0], [876.0, 0.6, 140.0], [669.0, 0.9, 304.0], [1036.0, 0.4, 204.0], [412.0, 0.9, 130.0], [945.0, 1.0, 315.0], [611.0, 0.6, 389.0], [825.0, 0.7, 354.0], [1487.0, 0.9, 102.0], [984.0, 0.9, 274.0], [1576.0, 0.4, 150.0], [797.0, 0.4, 256.0], [1555.0, 1.0, 343.0], [786.0, 1.1, 375.0], [1502.0, 0.5, 173.0], [830.0, 0.6, 369.0], [399.0, 1.1, 195.0]] 



In [56]:
numGenes = len(myList)-1 #кол-во грузов, т.е. генов (в одной особи 30 генов)
numInd = 200

In [57]:
def weight_checking(individual):
    sumWeight=0
    for i in range(numGenes):
        sumWeight += individual[i] * weight[i]
    if sumWeight > maxWeight:
        return 0
    else: 
        return sumWeight

def volume_checking(individual):
    sumSize=0
    for i in range(numGenes):
        sumSize += individual[i] * size[i]
    if sumSize > maxVolume:
        return 0
    else: 
        return sumSize

In [58]:
# Определяет стоимость набора, по сути fitness-функция
def countPrice(individual, data):
    weight, volume, price = 0, 0, 0
    for i in range(len(individual)):
        weight += int(data[i][0]) * individual[i]
        volume += float(data[i][1]) * individual[i]
        price += int(data[i][2]) * individual[i]
    if weight > maxWeight or volume > maxVolume:
        return 0
    return price

In [59]:
#Начальная популяция - 1.1 случайная генерация
#Сначала создаем особь, потом популяцию
def creation_of_individual():
    individual = [rd.randint(0, 1) for x in range(numGenes)] #особь
    while not (volume_checking(individual) and weight_checking(individual)):
        individual = [rd.randint(0, 1) for x in range(numGenes)]
    return individual


def creation_of_population():
    initial_population=[]
    for x in range(numInd):
        initial_population.append(creation_of_individual())
    return initial_population

In [60]:
# Выбор лучших 20%
def selection(myPopulation, data):
    valueOfPopulation = []
    for i in range(len(myPopulation)):
        individualList = []
        individualList.append(countPrice(myPopulation[i], data))
        individualList.append(myPopulation[i])
        valueOfPopulation.append(individualList)
    valueOfPopulation.sort(reverse=True)

    numberOfSelected = len(myPopulation) // 5
    selected = []
    for i in range(numberOfSelected):
        selected.append(valueOfPopulation[i][1])

    return selected

Скрещивание - 3.1 многоточечный с 3мя точками

In [61]:
# Получает двух новых особей на основе двух родительских
def findChildren(parent1, parent2):
    point1 = random.randint(0, len(parent1) // 3)
    point2 = random.randint(point1, len(parent1) // 3 * 2)
    point3 = random.randint(point2, len(parent1))

    child1 = parent1[0:point1] + parent2[point1:point2] + parent1[point2:point3] + parent2[point3:]
    child2 = parent2[0:point1] + parent1[point1:point2] + parent2[point2:point3] + parent1[point3:]

    return child1, child2

In [62]:
# Скрещивание(лучших детишек берем, худших меняем на родителей)
def crossing(selectedIndividuals, data): #особь
    parents = sorted(selectedIndividuals, key=lambda *args: random.random()) #возвращаем случайное значение
    numberOfPairs = len(selectedIndividuals) // 2 #делаем пары
    crossed = []
    for pairNum in range(numberOfPairs):
        child1, child2 = findChildren(parents[pairNum * 2], parents[pairNum * 2 + 1]) #скрещиваем 0 и 1, потом 2 и 3

        if countPrice(child1, data) != 0:
            crossed.append(child1)
        else:
            crossed.append(parents[pairNum * 2])

        if countPrice(child2, data) != 0:
            crossed.append(child2)
        else:
            crossed.append(parents[pairNum * 2 + 1])

    return crossed

In [63]:
# Мутация
def mutating(crossedIndividuals, data, numGenes):
    mutatedIndex = []
    numberOfMutations = len(crossedIndividuals) // 20 #наши 5 проц
    for mutatingNum in range(numberOfMutations): #добаляем в первом цикле первый индекс, while позволяет не записать такой же индекс второй раз
        tryIndex = random.randint(0, len(crossedIndividuals))
        while tryIndex in mutatedIndex:
            tryIndex = random.randint(0, len(crossedIndividuals))
        mutatedIndex.append(tryIndex)

        #выбираем, какие биты менять
        bitsIndex = []
        while len(bitsIndex) != 3:
            tryIndex = random.randint(0, numGenes - 1)
            if tryIndex in bitsIndex:
                continue
            else:
                bitsIndex.append(tryIndex)

        # print(crossedIndividuals[tryIndex])
        # сама мутация
        for i in range(len(bitsIndex)):
            crossedIndividuals[tryIndex][bitsIndex[i]] ^= 1 

    return crossedIndividuals

In [64]:
def makeNewPopulation(myPopulation, data, mutatedIndividuals):
    valueOfPopulation = []
    for i in range(len(myPopulation)):
        individualList = []
        individualList.append(int(countPrice(myPopulation[i], data) * 0.9)) #изменение и добавление ценности; у всей старой популяции забираем по 10%
        individualList.append(myPopulation[i])
        valueOfPopulation.append(individualList)

    for i in range(len(mutatedIndividuals)):
        individualMutatedList = []
        individualMutatedList.append(countPrice(mutatedIndividuals[i], data))
        individualMutatedList.append(mutatedIndividuals[i])
        valueOfPopulation.append(individualMutatedList)

    valueOfPopulation.sort(reverse=True)

    #добиваем до 200 осыбей
    while numInd  < len(valueOfPopulation):
        valueOfPopulation.pop(numInd )

    return valueOfPopulation

In [65]:
def genAlg(myPopulation, data, lowestCost, circles):
    circles += 1 #кол-во поколений
    forMaxVal = copy.deepcopy(myPopulation) #создаем полноую копию популяции, чтобы не копировать ссылку, а отдельный массив такой же

    valueOfPopulation = []
    for i in range(len(forMaxVal)):
        individualMutatedList = []
        individualMutatedList.append(countPrice(forMaxVal[i], data))
        individualMutatedList.append(forMaxVal[i])
        valueOfPopulation.append(individualMutatedList)
    valueOfPopulation.sort(reverse=True)

    maxVal = valueOfPopulation[0][0] #берем самую большую ценность

    while circles <= 500:
        # Отбор особей
        selectedIndividuals = selection(myPopulation, data)

        # Скрещивание друг с другом
        crossedIndividuals = crossing(selectedIndividuals, data)

        # Мутация 5%
        mutatedIndividuals = mutating(crossedIndividuals, data, numGenes)

        # Новая популяция, штраф старых
        newPopulation = makeNewPopulation(myPopulation, data, mutatedIndividuals)

        #разница между max ценностью старой и новой популяции
        if abs(maxVal - newPopulation[0][0]) < lowestCost and circles > 100:
            return newPopulation
        maxVal = newPopulation[0][0]
        circles += 1

    return newPopulation

In [66]:


# Генерируем популяцию
myPopulation = creation_of_population()
lowestCost = -1

file = open('11.txt')

line = file.readline()
restrictions = line.split()
maxWeight = int(restrictions[0])
maxVolume = int(restrictions[1])

data = []
line = file.readline()
lowestCost = -1
while line:
    data.append(line.split())
    line = file.readline()

for i in range(numGenes-1):
    price = int(data[i][2])
    if lowestCost == -1 or price < lowestCost:
        lowestCost = price

file.close()

finalPopulation = genAlg(myPopulation, data, lowestCost, 0)

weight, volume = 0, 0

for i in range(numGenes):
    weight += int(data[i][0]) * finalPopulation[0][1][i]
    volume += float(data[i][1]) * finalPopulation[0][1][i]

f = open('result.txt', 'w')
f.write("Value: " + str(finalPopulation[0][0]) + '\n')
f.write("Weight: " + str(weight) + '\n')
f.write("Volume: " + str(volume) + '\n')

print("Value: " + str(finalPopulation[0][0]))
print("Weight: " + str(weight))
print("Volume: " + str(volume))
f.close()

Value: 4046
Weight: 12275
Volume: 11.6
